# Build Stacks

In [ ]:
from pathlib import Path

import geopandas as gpd
import rasterio

from vegmapper import build_stack, build_condensed_stack

## User Inputs

In [ ]:
# Site name
sitename = 'ucayali'

# Project directory (local path or cloud bucket URL)
proj_dir = sitename

# AOI tiles
aoi_tiles = f'{proj_dir}/{sitename}_tiles.geojson'

# Resolution
res = 30

# Time
year = 2020
start_date = '2020-06-01'
end_date = '2020-07-31'

# Stack name & directory
stack_name = f'{sitename}_stacks_{year}'
stack_dir = f'{proj_dir}/stacks'

# Google Storage URL
gs = 'gs://YOUR_GS_URL'

# Google Earth Engine (GEE) project & ImageCollection ID
gee_project = f'projects/servir-amazonia'
collection_id = f'{sitename}_stack_{year}_test'

## Bands to be included in the stacks

The filename for each band is expected to be `{prefix}h{h_index}v{v_index}{suffix}`.

In [ ]:
bands = {
    '1': {
        'name': 'C-VV',
        'dir': f'{proj_dir}/Sentinel-1/S1B_{start_date}_{end_date}/vrt',
        'prefix': 'C-VV-',
        'suffix': '.vrt'
    },
    '2': {
        'name': 'C-VH',
        'dir': f'{proj_dir}/Sentinel-1/S1B_{start_date}_{end_date}/vrt',
        'prefix': 'C-VH-',
        'suffix': '.vrt'
    },
    '3': {
        'name': 'C-INC',
        'dir': f'{proj_dir}/Sentinel-1/S1B_{start_date}_{end_date}/vrt',
        'prefix': 'C-INC-',
        'suffix': '.vrt'
    },
    '4': {
        'name': 'L-HH',
        'dir': f'{proj_dir}/ALOS-2/mosaic',
        'prefix': 'L-HH-2020-2019-',
        'suffix': '.vrt'
    },
    '5': {
        'name': 'L-HV',
        'dir': f'{proj_dir}/ALOS-2/mosaic',
        'prefix': 'L-HV-2020-2019-',
        'suffix': '.vrt'
    },
    '6': {
        'name': 'L-INC',
        'dir': f'{proj_dir}/ALOS-2/mosaic',
        'prefix': 'L-INC-2020-2019-',
        'suffix': '.vrt'
    },
    '7': {
        'name': 'NDVI',
        'dir': f'{proj_dir}/landsat/{year}',
        'prefix': f'landsat_ndvi_{sitename}_{year}_',
        'suffix': '.tif'
    },
    '8': {
        'name': 'TC',
        'dir': f'{proj_dir}/modis/{year}',
        'prefix': f'modis_tc_{sitename}_{year}_',
        'suffix': '.tif'
    },
}

## Build stack for each tile

In [ ]:
# build_stack(stack_name, stack_dir, bands, aoi_tiles)

## Build condensed stacks

For quick data quality assessment and better visualization, we will build a "condensed" version of the stacks built above. The condensed stacks will be of `Int16` data type and will include the following four bands:

1. C-band Radar Vegetation Index x 100
2. L-band Radar Vegetation Index x 100 
3. NDVI x 100
4. Percent Tree Cover

The Radar Vegetation Index (RVI) is defined as:

$RVI = \frac{4CX}{CO + CX}$

where $CO$ and $CX$ are the co-polarized (VV or HH) and cross-polarized (VH or HV) radar backscatter in linear scale, respectively.

In [ ]:
condensed_stack_name = f'{sitename}_condensed_stacks_{year}'
build_condensed_stack(stack_name, condensed_stack_name, stack_dir)

## Upload condensed stacks to GEE

Before uploading images into the GEE destination ImageCollection, try to list it to see if it exists and what images have been included.

In [ ]:
# List the contents of ImageCollection on GEE
!earthengine ls $gee_project/$collection_id

If the ImageCollection does exist, make sure that you do want to upload/append images to the collection. If the ImageCollection doesn't exist, we need to create it before we can upload images to it.

In [ ]:
!earthengine create collection $gee_project/$collection_id

Now we first upload the condensed stacks to GS bucket and then request the images to be ingested to the collection. You can also check the status here: https://code.earthengine.google.com/tasks.

In [ ]:
stack_dir = Path(stack_dir)
condensed_stacks = sorted(stack_dir.glob(f'{condensed_stack_name}_h*v*.tif'))
for stack_tif in condensed_stacks:
    print(f'Uploading {stack_tif.name} to GEE ...')
    
    # Copy stack to GS
    gs_url = f'{gs}/{stack_tif.name}'
    !gsutil cp $stack_tif $gs_url
    
    # Upload stack to GEE
    image_name = stack_tif.stem
    !earthengine upload image --asset_id=$gee_project/$collection_id/$image_name -ts $start_date -te $end_date $gs_url